In [ ]:
# not strictly necessary, but imported anyway for the IDE haha
import torch
from torchvision import datasets, transforms 
from sklearn.metrics import classification_report

%run setup.ipynb 

# Change model setup 

Change learning rate and add weight decay 

In [ ]:
# Defining the transforms including augmentations (and also the basic ToTensor and normalisation)
preprocess_with_augmentation = transforms.Compose([
    transforms.ColorJitter(brightness=0.1),
    transforms.GaussianBlur(kernel_size=(5,5), sigma=(7, 9)),
    transforms.RandomRotation(degrees=10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomResizedCrop(size=224,scale=(0.3,1)), # lowerbound the scale at 30 % of og img to not get too small portions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Datasets and loaders with augmentations added 
dataset_train_aug = datasets.ImageFolder(PATH_TRAIN, preprocess_with_augmentation)
dataloader_train_aug = torch.utils.data.DataLoader(dataset_train_aug, batch_size=8, shuffle=True, num_workers=4)

In [ ]:
# Set and load model
untrained_model_aug = load_and_prep_resnet50()
optimizer = torch.optim.SGD(untrained_model_aug.parameters(), lr=0.02, momentum=0.9, weight_decay=0.00001)

In [ ]:
freezetrained_model_aug, _ = train_model(model=untrained_model_aug, optimizer=optimizer, num_epoch=24, train=dataloader_train_aug) 

In [ ]:
unfreeze_layers(freezetrained_model_aug)
model_aug, model_aug_info = train_model(model=freezetrained_model_aug, optimizer=optimizer, num_epoch=24, train=dataloader_train_aug)

## Results

In [ ]:
visualize_model(model_aug, dataloader_tiny_val, class_labels, device)

In [ ]:
plot_accuracies(model_aug_info["train_loss"], model_aug_info["val_loss"], ["Train", "Val"])

In [ ]:
plot_accuracies(model_aug_info["train_acc"], model_aug_info["val_acc"], ["Train", "Val"])

In [ ]:
truel, predl = makeAll(dataloader_test, model_aug, device)
plot_confusion_matrix(truel, predl, class_labels, normalize=True)
print("Accuracy", (np.sum(predl==truel)/predl.size * 100))

In [ ]:
class_labels = ['blueTit', 'chaffinch', 'coalTit', 'goldfinch', 'greatTit', 'robin', 'starling']
print(classification_report(truel, predl, target_names=class_labels))